In [1]:
import pandas as pd
data = pd.read_csv('data/posfinal_14.csv')

In [2]:
import numpy as np

In [3]:
data = data.assign(width= lambda x: x['maxpos'] - x['minpos'],
                  center= lambda x: (x['maxpos'] + x['minpos'])/2,
                  cursize= lambda x: np.where(((x['maxpos'] == x['curpos'])|(x['minpos']==x['curpos'])),20,10))

In [4]:
from bokeh.io import output_file, output_notebook, show

In [5]:
output_notebook()

Loading BokehJS ...

In [6]:
# output_file('posrange.html')

In [7]:
from bokeh.models import (ColumnDataSource, 
                          Plot, 
                          LinearAxis,
                          CategoricalAxis,
                          Range1d, 
                          FactorRange, 
                          Circle,
                          Label,
                          FixedTicker,
                          Grid,
                          Rect,
                          LabelSet,
                          Legend
                          )
# from bokeh.plotting import Figure

In [8]:
cds = ColumnDataSource(data)

p = Plot(title=None, plot_width=500, plot_height=600, min_border_top=70, 
         toolbar_location=None, logo=None, outline_line_color=None,
          x_range=Range1d(0,21), y_range=FactorRange(factors=list(data.team)[::-1]))

yax = CategoricalAxis(major_label_text_align='left', axis_line_color=None, 
                     major_label_text_font_size='16px', major_label_text_font_style='bold',
                     major_tick_line_color=None)
p.add_layout(yax, 'left')

p.add_glyph(cds, Rect(x='center', y='team',
                     width='width', height=0.2, 
                     line_color=None, fill_color='#9D9D9D' ))

maxp = p.add_glyph(cds, Circle(x='maxpos',y='team',fill_color='#75C5E4',line_color=None, size=20))
minp = p.add_glyph(cds, Circle(x='minpos',y='team',fill_color='#0068A1',line_color=None, size=20))
curp = p.add_glyph(cds, Circle(x='curpos',y='team',fill_color='#9D9D9D',line_color=None, size='cursize'))

labarg = dict(render_mode='canvas', text_align='center', 
                  text_baseline='middle', text_font_size='8pt',
                  text_color='#FFFFFF', text_font_style='bold', 
                  x_offset=-0.5, y_offset=-1, source=cds)

minlab = LabelSet(x='minpos', y='team', text='minpos', **labarg)
maxlab = LabelSet(x='maxpos', y='team', text='maxpos', **labarg)
p.add_layout(minlab)
p.add_layout(maxlab)

ticky = FixedTicker(ticks=[4.5, 10.5, 17.5])
tickx = FixedTicker(ticks=[16.5, 10.5, 3.5])
grx = Grid(dimension=1, ticker=tickx)
gry = Grid(dimension=0, ticker=ticky)
p.add_layout(grx)
p.add_layout(gry)

labarg2 = dict(x=5, y=10, x_units='screen', y_units='screen')
subhead1 = Label(text='Spread of possible positions following fixtures on December 3-5;',
                **labarg2)

# subhead2 = Label(text='Thin grey lines indicate top four, top half and relegation zone',
#                 **labarg2)

head = Label(text_font_style='bold', text_font_size='20px', 
             text='How much could the Premier League table change?',
            **labarg2)

# p.add_layout(subhead2, 'above')
p.add_layout(subhead1, 'above')
p.add_layout(head, 'above')

leg = Legend(items=[
        ('highest possible rise', [minp]),
        ('lowest possible fall', [maxp]),
        ('current position', [curp])
    ], orientation='horizontal',
            location='bottom_center')
p.add_layout(leg, 'below')

show(p)